In [44]:
import pandas as pd
from time import sleep


# Liste des fichiers à analyser
datafiles = ['250101_Liste_fonds_label_ISR-6.xlsx', '250101_Liste_fonds_label_ISR-7.xlsx']

# Liste des mots-clés à rechercher
keywords = ['inclusion', 'diversity', 'emploi', 'health', 'santé', 'equity']

# Initialiser une liste pour stocker les résultats
total_matching_cells = []

for datafile in datafiles:
    # Charger le fichier Excel
    df = pd.read_excel(datafile)
    
    # Sélectionner la deuxième colonne (B), peu importe son nom réel
    col_b_name = df.columns[1]  # La deuxième colonne a l'index 1
    
    # Filtrer les cellules contenant au moins un des mots-clés
    matching_cells = df[df[col_b_name].astype(str).str.contains('|'.join(keywords), case=False, na=False)][col_b_name].tolist()
    
    # Ajouter les résultats avec le nom du fichier comme référence
    total_matching_cells.extend([(datafile, cell) for cell in matching_cells])

# Convertir en DataFrame et sauvegarder le résultat
output_df = pd.DataFrame(total_matching_cells, columns=['Fichier', 'Nom du Fonds'])
output_file = 'matching_cells.xlsx'
output_df.to_excel(output_file, index=False)

print(f"Résultats enregistrés dans {output_file}")

Résultats enregistrés dans matching_cells.xlsx


In [3]:
pip install pandas openpyxl



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [42]:
import os
import pandas as pd

def lire_txt_en_dataframe(chemin_fichier):
    # Lire les données (séparateur = tabulation)
    df = pd.read_csv(chemin_fichier, sep='\t')

    # Garder seulement les colonnes "date" et "close"
    df = df.iloc[:, [0, 4]]
    df.columns = ['date', 'close']

    # Convertir les dates en datetime
    df['date'] = pd.to_datetime(df['date'], dayfirst=True)
    return df

def merge_fichiers_avec_isin(fichiers, mapping_isin, dossier_output='financial_data', nom_fichier='data_fonds.csv'):
    merged_df = None

    # Créer le dossier s'il n'existe pas
    os.makedirs(dossier_output, exist_ok=True)

    for fichier in fichiers:
        isin = mapping_isin.get(fichier)
        if not isin:
            raise ValueError(f"Aucun ISIN trouvé pour le fichier : {fichier}")

        df = lire_txt_en_dataframe(fichier)
        df = df.rename(columns={'close': isin})

        if merged_df is None:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on='date', how='outer')

    # Trier par date et réinitialiser l'index
    merged_df = merged_df.sort_values('date').reset_index(drop=True)

    # Construire le chemin de sortie complet
    chemin_csv = os.path.join(dossier_output, nom_fichier)

    # Sauvegarder le dataframe dans un fichier CSV
    merged_df.to_csv(chemin_csv, index=False)

In [43]:
fichiers = [
    'INSERTIONEMPLOISDYNAMIQUERC_2025-04-09.txt',
    'CANDRIAMBONDSEUROCORPORATE_2025-04-12.txt'
]

# Dictionnaire de mapping : nom du fichier => ISIN
mapping_isin = {
    'INSERTIONEMPLOISDYNAMIQUERC_2025-04-09.txt': '0P0000KU3M.F',
    'CANDRIAMBONDSEUROCORPORATE_2025-04-12.txt': 'LU1313770536'
}

merge_fichiers_avec_isin(fichiers, mapping_isin)
merge_fichiers_avec_isin


<function __main__.merge_fichiers_avec_isin(fichiers, mapping_isin, dossier_output='financial_data', nom_fichier='data_fonds.csv')>